In [1]:
import pandas as pd
import numpy as np


In [2]:
!ls datasets/housing

housing.csv  housing.tgz  README.md


In [4]:
housing=pd.read_csv("./datasets/housing/housing.csv")
# housing.head()

In [5]:
# find categrocal_col
# housing.describe(include=['O'])

In [6]:
# housing.corr()['median_house_value'].sort_values(ascending=False)

<b>so median income and bedroom per total room effect house value</b>

In [7]:
housing['bedroom_per_total_room']=housing.total_rooms/housing.total_bedrooms

In [8]:
# based on info just total bedroom and bedroom per room have missing values
col_with_missing=list(housing.columns[housing.isnull().any()])
print("col with missing value",col_with_missing)
# housing.info()


col with missing value ['total_bedrooms', 'bedroom_per_total_room']


In [9]:
# housing.corr()['median_house_value'].sort_values(ascending=False)

In [11]:
from pandas.plotting import scatter_matrix
feature=['total_rooms','total_bedrooms','median_house_value','median_income']
# scatter_matrix(housing[feature],figsize=(12,12))

In [12]:
# housing.corr()['total_bedrooms'].sort_values()

so total bedroom have strong relationship with households and totalroom and populations

In [13]:
from sklearn.model_selection import train_test_split
X=housing.copy().drop(labels='median_house_value',axis=1)
y=housing.median_house_value.copy()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

build pipeline

In [50]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
# impute=IterativeImputer(random_state=42)
# numerical_transformer=StandardScaler()
#we dont have missing value in categorical number so we dont need imputer
#only ocean_proximity is categorical
categorical_col=X_train.select_dtypes(include='object_').columns
numerial_col=X_train.select_dtypes(exclude='object_').columns
# categorical_transformer=OneHotEncoder(handle_unknown='ignore')
preprocsssor=ColumnTransformer(transformers=[('impute',SimpleImputer(strategy='median'),col_with_missing),
                                             ('normalize',StandardScaler(),numerial_col.drop(col_with_missing)),
                                             ('cat',OneHotEncoder(handle_unknown='ignore'),categorical_col)],n_jobs=8)
# ('normalize',StandardScaler(),numerial_col),


# we can't use normalize with cols have missing value

In [46]:
# preprocsssor.fit(X_train)
X_train_tr=preprocsssor.fit_transform(X_train)
X_test_tr=preprocsssor.transform(X_test)
# X_test_tr.dropna(axis=1,inplace=True)
# X_train.columns


biuld model

In [47]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(X_train_tr,y_train)
# my_pipeline=Pipeline(steps=[('preprocsssor',preprocsssor),
#                             ('model',DecisionTreeRegressor())])
# my_pipeline.fit(X_train_tr,y_train)

RandomForestRegressor()

get_score

In [48]:
from sklearn.metrics import mean_squared_error
y_hat=model.predict(X_test_tr)
score=mean_squared_error(y_test,y_hat)
score

2411144950.5853825

In [49]:
# for col in X_test_tr.col
# X_test_tr.isnull()
type(X_test_tr)

numpy.ndarray

In [52]:
from xgboost import XGBRegressor
XG=XGBRegressor(n_jobs=8)
XG.fit(X_train_tr,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [53]:
y_hat=XG.predict(X_test_tr)
score=mean_squared_error(y_test,y_hat)
score

2178861430.364649

In [54]:
from sklearn.model_selection import cross_validate
X_tr=preprocsssor.fit_transform(X)
scores=cross_validate(XG,X_tr,y,scoring="neg_mean_squared_error",cv=10)


In [59]:
from sklearn.model_selection import GridSearchCV
param_grid = [
 {'n_estimators': [10, 100, 1000]},
 {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
 ]
grid_search=GridSearchCV(XG,param_grid=param_grid,cv=5,return_train_score=True,n_jobs=8)
grid_search.fit(X_tr,y)

[00:02:43] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:02:43] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:02:43] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you 

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0, gpu_id=-1,
                                    importance_type='gain',
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=8,
                                    num_parallel_tree=1, random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', validate_parameters=1,
                                    verbosity=None),
      